# ⚙️ Configuração de Ambiente dbt com DuckDB

Este notebook automatiza a criação e configuração de um ambiente `dbt` com `DuckDB` usando ambiente virtual Python.

**Etapas:**
1. Criação de ambiente virtual
2. Instalação de dependências
3. Criação de banco DuckDB
4. Inicialização de projeto dbt
5. Criação do arquivo de perfil do dbt
6. Teste final da configuração

## 0- Imports gerais

In [2]:
import os
import sys
import subprocess
import venv
import json
from pathlib import Path

## 1. Criação de ambiente virtual

In [8]:
def create_and_activate_venv():
    venv_name = ".venv"
    if os.path.exists(venv_name):
        print(f"✅ Ambiente virtual '{venv_name}' já existe.")
    else:
        print(f"🔧 Criando ambiente virtual '{venv_name}'...")
        venv.create(venv_name, with_pip=True)
        print("✅ Ambiente virtual criado com sucesso!")

    if sys.platform == "win32":
        python_path = os.path.join(venv_name, "Scripts", "python.exe")
        pip_path = os.path.join(venv_name, "Scripts", "pip.exe")
    else:
        python_path = os.path.join(venv_name, "bin", "python")
        pip_path = os.path.join(venv_name, "bin", "pip")
    return python_path, pip_path

python_path, pip_path = create_and_activate_venv()

✅ Ambiente virtual '.venv' já existe.


## 2. Instalação de dependências

In [ ]:
def install_packages(pip_path):
    packages = ["dbt-duckdb", "pandas", "duckdb"]
    for package in packages:
        print(f"📦 Instalando {package}...")
        subprocess.run([pip_path, "install", package], check=True)

install_packages(pip_path)

📦 Instalando dbt-duckdb...
📦 Instalando pandas...
📦 Instalando duckdb...


### 3. Inicialização de projeto dbt

```
dbt init meu_projeto_dbt
```

In [7]:
def create_model_directories():
    base_path = "meu_projeto_dbt/models"
    directories = ["raw", "staging", "intermediate", "mart"]
    for d in directories:
        dir_path = os.path.join(base_path, d)
        os.makedirs(dir_path, exist_ok=True)
        init_file = os.path.join(dir_path, "__init__.py")
        if not os.path.exists(init_file):
            with open(init_file, 'w') as f:
                f.write(f"# {d} models\n")
create_model_directories()

### 4. Inicialização de projeto dbt

In [6]:
import duckdb
def create_duckdb_database():
    db_path = "bd/dev.duckdb"
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    conn = duckdb.connect(db_path)
    result = conn.execute("SELECT 'Conexão estabelecida com sucesso!' as status").fetchone()
    print(f"🗄️ Banco DuckDB criado/verificado: {db_path}")
    print(f"📊 Status: {result[0]}")
    conn.close()
    return db_path

create_duckdb_database()

🗄️ Banco DuckDB criado/verificado: bd/dev.duckdb
📊 Status: Conexão estabelecida com sucesso!


'bd/dev.duckdb'

#### 4.1 Teste de conexão ao banco

In [8]:
def test_database_connection():
    conn = duckdb.connect("meu_projeto_dbt/dev.duckdb")
    result = conn.execute("SELECT version() as duckdb_version").fetchone()
    print(f"🔌 Conexão OK - Versão DuckDB: {result[0]}")
    conn.execute("CREATE TABLE IF NOT EXISTS teste_conexao (id INTEGER, mensagem VARCHAR)")
    conn.execute("INSERT INTO teste_conexao VALUES (1, 'Teste ok')")
    print(conn.execute("SELECT * FROM teste_conexao").fetchall())
    conn.execute("DROP TABLE teste_conexao")
    conn.close()
test_database_connection()

🔌 Conexão OK - Versão DuckDB: v1.3.2
[(1, 'Teste ok')]


###  5. Criação do arquivo de perfil do dbt

In [9]:
try:
    import yaml
except ImportError:
    yaml = None

def create_dbt_profile():
    profile_dir = os.path.expanduser("~/.dbt")
    os.makedirs(profile_dir, exist_ok=True)
    profiles_path = os.path.join(profile_dir, "profiles.yml")
    config = {
        "meu_projeto_dbt": {
            "target": "dev",
            "outputs": {
                "dev": {
                    "type": "duckdb",
                    "path": os.path.abspath("meu_projeto_dbt/dev.duckdb"),
                    "schema": "main"
                }
            }
        }
    }
    with open(profiles_path, 'w') as f:
        if yaml:
            yaml.dump(config, f)
        else:
            json.dump(config, f, indent=2)
    print(f"✅ Perfil dbt criado em: {profiles_path}")
create_dbt_profile()

✅ Perfil dbt criado em: C:\Users\abili/.dbt\profiles.yml


### 6. Teste final da configuração

In [10]:
def test_dbt_configuration():
    os.chdir("meu_projeto_dbt")
    dbt_path = os.path.join("..", ".venv", "Scripts" if sys.platform == "win32" else "bin", "dbt")
    result = subprocess.run([dbt_path, "debug"], capture_output=True, text=True)
    print(result.stdout)
    os.chdir("..")
test_dbt_configuration()

16:57:02  Running with dbt=1.10.10
16:57:02  dbt version: 1.10.10
16:57:02  python version: 3.12.10
16:57:02  python path: c:\Users\abili\Documents\GitHub\introducao_dbt\.venv\Scripts\python.exe
16:57:02  os info: Windows-11-10.0.26100-SP0
16:57:02  Using profiles dir at C:\Users\abili\.dbt
16:57:02  Using profiles.yml file at C:\Users\abili\.dbt\profiles.yml
16:57:02  Using dbt_project.yml file at c:\Users\abili\Documents\GitHub\introducao_dbt\meu_projeto_dbt\dbt_project.yml
16:57:02  adapter type: duckdb
16:57:02  adapter version: 1.9.4
16:57:02  Configuration:
16:57:02    profiles.yml file [OK found and valid]
16:57:02    dbt_project.yml file [OK found and valid]
16:57:02  Required dependencies:
16:57:02   - git [OK found]

16:57:02  Connection:
16:57:02    database: dev
16:57:02    schema: main
16:57:02    path: bd\dev.duckdb
16:57:02    config_options: None
16:57:02    extensions: None
16:57:02    settings: {}
16:57:02    external_root: .
16:57:02    use_credential_provider: None
